## **Red Black Tree**

In [31]:
from time import time
from random import randint

In [32]:
class Node:
    def __init__(self,data,red=True):
        self.data = data
        self.left = None
        self.right = None
        self.parent = None
        self.red = red

In [33]:
class RedBlackTree:
    def __init__(self):
        self.root = Node(None,False)
        self.root.left = Node(None,False)
        self.root.right = Node(None,False)

    def insert(self,data):
        node = self.newNode(data)

        root = self.root
        temp = None

        while root.data is not None:
            temp = root
            if node.data < root.data:
                root = root.left
            else:
                root = root.right
        
        node.parent = temp
        if temp is None:
            self.root = node
        elif node.data < temp.data:
            temp.left = node
        else:
            temp.right = node
        
        if node.parent is None:
            node.red = False
            return

        if node.parent.parent is None:
            return

        self.fixInsertion(node)
        

    def fixInsertion(self,node):
        while node.parent.red:
            parent = node.parent
            grandparent = node.parent.parent
            if grandparent.left == parent:
                uncle = grandparent.right
                # recolor
                if uncle.red:
                    parent.red = False
                    uncle.red = False
                    grandparent.red = True
                    node = grandparent
                # if triangle, create line
                else:
                    if node == parent.right:
                        node = node.parent
                        self.leftRotate(node)
                    node.parent.red = False
                    node.parent.parent.red = True
                    self.rightRotate(node.parent.parent)
            
            else:
                uncle = grandparent.left
                if uncle.red:
                    parent.red = False
                    uncle.red = False
                    grandparent.red = True
                    node = grandparent
                else:
                    if node == parent.left:
                        node = node.parent
                        self.rightRotate(node)
                    node.parent.red = False
                    node.parent.parent.red = True
                    self.leftRotate(node.parent.parent)

            if node == self.root or node.parent == self.root:
                break

        self.root.red = False
        

        ### Recursion Version
        # if node == self.root:
        #     node.red = False
        #     return node
        
        # parent = node.parent
        # if parent.red:
        #     # case 0: make root black
        #     if parent == self.root:
        #         parent.red = False
        #         return node
        
        #     grandparent = parent.parent
        #     uncle = None

        #     if parent == grandparent.left:
        #         uncle = grandparent.right
        #         # case 1: recolor
        #         if uncle.red:
        #             print("uncle red left")
        #             parent.red = False
        #             uncle.red = False
        #             grandparent.red = True
        #             self.fixInsertion(grandparent)
        #         # case 2: triangle
        #         elif parent.right == node:
        #             print("triangle left")
        #             self.leftRotate(parent)
        #             self.fixInsertion(grandparent.left.left)
        #         # line
        #         else:
        #             print("line left")
        #             parent.red = False
        #             grandparent.red = True
        #             self.rightRotate(grandparent)

        #     else:
        #         uncle = grandparent.left
        #         if uncle.red:
        #             print("uncle red right")
        #             parent.red = False
        #             uncle.red = False
        #             grandparent.red = True
        #             self.fixInsertion(grandparent)
                
        #         elif parent.left == node:
        #             print("triangle right")
        #             self.rightRotate(parent)
        #             self.fixInsertion(grandparent.right.right)
                
        #         else:
        #             print("line right")
        #             parent.red = False
        #             grandparent.red = True
        #             self.leftRotate(grandparent)
        # return node



    def leftRotate(self, root):
        y = root.right
        root.right = y.left

        if y.left.data is not None:
            y.left.parent = root
        
        y.parent = root.parent

        if root.parent is None:
            self.root = y
        elif root == root.parent.left:
            root.parent.left = y
        else:
            root.parent.right = y
        y.left = root
        root.parent = y

    def rightRotate(self, root):
        y = root.left
        root.left = y.right
        if y.right.data is not None:
            y.right.parent = root

        y.parent = root.parent

        if root.parent == None:
            self.root = y
        elif root == root.parent.right:
            root.parent.right = y
        else:
            root.parent.left = y
            
        y.right = root
        root.parent = y
    
    # new node has 2 black nil children
    def newNode(self, data):
        node = Node(data,True)
        node.left = Node(None,False)
        node.right = Node(None,False)
        return node
    

    def delete(self,target):
        root = self.root
        # empty tree
        if root.data is None:
            return
        
        # search for target
        delete_node = None
        while root.data is not None:
            if target == root.data:
                delete_node = root
                break
            elif target < root.data:
                root = root.left
            else:
                root = root.right
        
        # not found
        if not delete_node:
            return
        
        original_color = delete_node.red
        # x replaces deleted node
        x = None

        # only has right child
        if delete_node.left.data is None:
            x = delete_node.right
            self.transplant(delete_node,x)
        # only has left child
        elif delete_node.right.data is None:
            x = delete_node.left
            self.transplant(delete_node,x)
        # has both children
        else:
            # get minimum value of left subtree
            min_node = delete_node.right
            while min_node.left.data is not None:
                min_node = min_node.left

            original_color = min_node.red
            x = min_node.right
            if min_node.parent == delete_node:
                x.parent = min_node
            else:
                self.transplant(min_node,x)
                min_node.right = delete_node.right
                min_node.right.parent = min_node

            self.transplant(delete_node,min_node)
            min_node.left = delete_node.left
            min_node.left.parent = min_node
            min_node.red = delete_node.red
        
        # if black node deleted
        if not original_color:
            self.delete_fixup(x)

    def transplant(self,x,y):
        if x.parent is None:
            self.root = y
        elif x.parent.left == x:
            x.parent.left = y
        else:
            x.parent.right = y
        y.parent = x.parent 
        

    def delete_fixup(self,x):
        while x != self.root and not x.red:
            if x == x.parent.left:
                sibling = x.parent.right
                # case 1: red sibling
                if sibling.red:
                    sibling.red = False
                    x.parent.red = True
                    self.leftRotate(x.parent)
                    sibling = x.parent.right

                # case 2: both sibling children are black
                if not sibling.left.red and not sibling.right.red:
                    sibling.red = True
                    x = x.parent
                else:
                    # case 3: right sibling black
                    if not sibling.right.red:
                        sibling.left.red = False
                        sibling.red = True
                        self.rightRotate(sibling)
                        sibling = x.parent.right
                    
                    # case 4: right sibling red
                    sibling.red = x.parent.red
                    x.parent.red = False
                    sibling.right.red = False
                    self.leftRotate(x.parent)
                    x = self.root
            else:
                sibling = x.parent.left
                if sibling.red:
                    sibling.red = False
                    x.parent.red = True
                    self.rightRotate(x.parent)
                    sibling = x.parent.left
                
                if not sibling.right.red and not sibling.left.red:
                    sibling.red = True
                    x = x.parent
                else:
                    if not sibling.left.red:
                        sibling.right.red = False
                        sibling.red = True
                        self.leftRotate(sibling)
                        sibling = x.parent.left
                    
                    sibling.red = x.parent.red
                    x.parent.red = False
                    sibling.left.red = False
                    self.rightRotate(x.parent)
                    x = self.root

        self.root.red = False

    def inOrder(self, root):
        if not root or not root.data:
            return
        self.inOrder(root.left)
        print(root.data, root.red)
        self.inOrder(root.right)
        
        

Insertion Test

In [38]:
N = 10000


for test in range(5):

    rb_tree = RedBlackTree()

    start = time()
    for i in range(1,N):
        rb_tree.insert(randint(1,1000))
    end = time()

    print(f"{end-start:.8f}")

# rb_tree.inOrder(rb_tree.root)

0.13085651
0.13057709
0.12880135
0.12975478
0.22237992


Deletion Test

In [35]:
N = 100000

nums = [x for x in range(N)]


for test in range(5):

    rb_tree = RedBlackTree()

    
    for i in range(1,N):
        rb_tree.insert(i)
    

    start = time()
    for i in range(1,N):
        rb_tree.delete(i)
    end = time()

    print(f"{end-start:.8f}")

0.59580994
0.59859228
0.60129738
0.59576893
0.59516621
